# Polars - V1 Report

[polars](https://www.pola.rs/)

The goal of Polars is to provide a lightning fast DataFrame library, that:

- Utilizes all available cores on your machine.
- Optimizes queries to reduce unneeded work/memory allocations.
- Handles datasets much larger than your available RAM.
- Has an API that is consistent and predictable.
- Has a strict schema (data-types should be known before running the query).
- Polars is written in Rust which gives it C/C++ performance and allows it to fully control performance critical parts in a query engine.

As such Polars goes to great lengths to:

- Reduce redundant copies.
- Traverse memory cache efficiently.
- Minimize contention in parallelism.
- Process data in chunks.
- Reuse memory allocations.
ry allocations.

## Imports

In [1]:
import polars as pl
import pandas as pd
import numpy as np

## Basic Polars

Like Pandas, Polars is composed of series and dataframe data structures. Unlike pandas, printing the structures shows the format of the data - i64 etc.

The data structures respect the use of the common utility functios such as head, tail, describe and sample.

In [2]:
pl.Series("a", [1, 2, 3, 4, 5])

a
i64
1
2
3
4
5


In [3]:
from datetime import datetime

pl.DataFrame(
    {
        "integer": [1, 2, 3, 4, 5],
        "date": [
            datetime(2022, 1, 1),
            datetime(2022, 1, 2),
            datetime(2022, 1, 3),
            datetime(2022, 1, 4),
            datetime(2022, 1, 5),
        ],
        "float": [4.0, 5.0, 6.0, 7.0, 8.0],
    }
)

integer,date,float
i64,datetime[μs],f64
1,2022-01-01 00:00:00,4.0
2,2022-01-02 00:00:00,5.0
3,2022-01-03 00:00:00,6.0
4,2022-01-04 00:00:00,7.0
5,2022-01-05 00:00:00,8.0


Polars has developed its own Domain Specific Language (DSL) for transforming data. The language is very easy to use and allows for complex queries that remain human readable. The two core components of the language are Contexts and Expressions

In [4]:
df = pl.DataFrame(
    {
        "nrs": [1, 2, 3, None, 5],
        "names": ["foo", "ham", "spam", "egg", None],
        "random": np.random.rand(5),
        "groups": ["A", "A", "B", "C", "B"],
    }
)

In the select context the selection applies expressions over columns. The expressions in this context must produce Series that are all the same length or have a length of 1.

A Series of a length of 1 will be broadcasted to match the height of the DataFrame. Note that a select may produce new columns that are aggregations, combinations of expressions, or literals.

In [5]:
df.select(
    pl.sum("nrs"),
    pl.col("names").sort(),
    pl.col("names").first().alias("first name"),
    (pl.mean("nrs") * 10).alias("10xnrs"),
)

nrs,names,first name,10xnrs
i64,str,str,f64
11,null,"""foo""",27.5
11,"""egg""","""foo""",27.5
11,"""foo""","""foo""",27.5
11,"""ham""","""foo""",27.5
11,"""spam""","""foo""",27.5


In [6]:
df.with_columns(
    pl.sum("nrs").alias("nrs_sum"),
    pl.col("random").count().alias("count"),
)

nrs,names,random,groups,nrs_sum,count
i64,str,f64,str,i64,u32
1,"""foo""",0.020161,"""A""",11,5
2,"""ham""",0.38525,"""A""",11,5
3,"""spam""",0.556443,"""B""",11,5
null,"""egg""",0.033624,"""C""",11,5
5,null,0.267556,"""B""",11,5


In the filter context you filter the existing dataframe based on arbritary expression which evaluates to the Boolean data type.

In [7]:
df.filter(pl.col("nrs") > 2)

nrs,names,random,groups
i64,str,f64,str
3,"""spam""",0.556443,"""B"""
5,null,0.267556,"""B"""


In the groupby context expressions work on groups and thus may yield results of any length (a group may have many members).

In [8]:
df.groupby("groups").agg(
    pl.sum("nrs"),  # sum nrs by groups
    pl.col("random").count().alias("count"),  # count group members
    # sum random where name != null
    pl.col("random").filter(pl.col("names").is_not_null()).sum().suffix("_sum"),
    pl.col("names").reverse().alias(("reversed names")),
)

groups,nrs,count,random_sum,reversed names
str,i64,u32,f64,list[str]
"""A""",3,2,0.40541,"[""ham"", ""foo""]"
"""B""",8,2,0.556443,"[null, ""spam""]"
"""C""",null,1,0.033624,"[""egg""]"


## Lazy v Eager API

Polars supports two modes of operation: lazy and eager. In the eager API the query is executed immediately while in the lazy API the query is only evaluated once it is 'needed'. Deferring the execution to the last minute can have significant performance advantages that is why the Lazy API is preferred in most cases.

In [9]:
df = pl.read_csv("iris.csv")
df_small = df.filter(pl.col("sepal_length") > 5)
df_agg = df_small.groupby("species").agg(pl.col("sepal_width").mean())

In this example we use the eager API to:

- Read the iris dataset.
- Filter the dataset based on sepal length
- Calculate the mean of the sepal width per species

Every step is executed immediately returning the intermediate results. This can be very wasteful as we might do work or load extra data that is not being used. If we instead used the lazy API and waited on execution until all the steps are defined then the query planner could perform various optimizations. In this case:

Predicate pushdown: Apply filters as early as possible while reading the dataset, thus only reading rows with sepal length greater than 5.

Projection pushdown: Select only the columns that are needed while reading the dataset, thus removing the need to load additional columns (e.g. petal length & petal width)l width)

In [10]:
q = (
    pl.scan_csv("iris.csv")
    .filter(pl.col("sepal_length") > 5)
    .groupby("species")
    .agg(pl.col("sepal_width").mean())
)

df = q.collect()
df

species,sepal_width
str,f64
"""Versicolor""",2.804255
"""Virginica""",2.983673
"""Setosa""",3.713636


It is also possible to swap out read_csv for scan_csv directly in code, rather than using the "q" structure, it is merly a grouping mechanism.

In [11]:
q.explain()

'AGGREGATE\n\t[col("sepal_width").mean()] BY [col("species")] FROM\n\n    CSV SCAN iris.csv\n    PROJECT 3/5 COLUMNS\n    SELECTION: [(col("sepal_length")) > (5.0)]'

These will significantly lower the load on memory & CPU thus allowing you to fit bigger datasets in memory and process faster. Once the query is defined you call collect to inform Polars that you want to execute it. 

## Streaming API

One additional benefit of the lazy API is that it allows queries to be executed in a streaming manner. Instead of processing the data all-at-once Polars can execute the query in batches allowing you to process datasets that are larger-than-memory.

To tell Polars we want to execute a query in streaming mode we pass the streaming=True argument to colle.

Streaming is still in development. We can ask Polars to execute any lazy query in streaming mode. However, not all lazy operations support streaming. If there is an operation for which streaming is not supported Polars will run the query in non-streaming mode.ct

In [12]:
q = (
    pl.scan_csv("iris.csv")
    .filter(pl.col("sepal_length") > 5)
    .groupby("species")
    .agg(pl.col("sepal_width").mean())
)

df = q.collect(streaming=True)

## Pandas v Polars, Syntax

[Source](https://pola-rs.github.io/polars-book/user-guide/migration/pandas/#pandas-transform)

### Selecting Columns

#### Pandas

df['a']

df.loc[:,'a']

#### Polars

df.select('a')

### Selecting conditional rows

#### Polars

df.filter(pl.col('a') < 10)

### Column Assignment

#### Pandas

df["tenXValue"] = df["value"] * 10

df["hundredXValue"] = df["value"] * 100

#### Polar

df.with_columns( (pl.col("value") * 10).alias("tenXValue")  (pl.col("value") * 100).alias("hundredXValue,
)s

a

## Pandas v Polars, Performance

This is a apples to apples test in data processing, it does not conside the vast benefits from using lazy execution methods

In [13]:
import time

# Create a sample dataframe
n = 200000000
df_pandas = pd.DataFrame({'A': range(n), 'B': range(n, 0, -1)})

# Convert the Pandas dataframe to a Polars dataframe
df_polars = pl.DataFrame(df_pandas)

# Benchmark Pandas
start_time = time.time()
# Perform a computation task on Pandas dataframe (e.g., sorting)
df_pandas.sort_values('B', inplace=True)
pandas_time = time.time() - start_time

# Benchmark Polars
start_time = time.time()
# Perform the same computation task on Polars dataframe
df_polars = df_polars.sort("B")
polars_time = time.time() - start_time

print("Pandas execution time: {:.3f} seconds".format(pandas_time))
print("Polars execution time: {:.3f} seconds".format(polars_time))

Pandas execution time: 31.663 seconds
Polars execution time: 16.005 seconds


## Key Findings

- PRO: Polars is built upon the safe Arrow2 implementation of the [Apache Arrow specification](https://arrow.apache.org/docs/format/Columnar.html), enabling efficient resource use and processing performance. By doing so it also integrates seamlessly with other tools in the Arrow ecosystem.
- PRO: Polars supports eager evaluation and lazy evaluation whereas Pandas only supports eager evaluation.
- CON: Polars is still developing, it may lack some functional operations avaliable in pandas.
- CON: The syntax and use of Polars differs enough from Pandas to have a moderate learning curve.
- PRO: Polars uses Apache Arrow arrays to represent data in memory while Pandas uses Numpy arrays
Polars represents data in memory with Arrow arrays while Pandas represents data in memory with Numpy arrays. Apache Arrow is an emerging standard for in-memory columnar analytics that can accelerate data load times, reduce memory usage and accelerate calculations.
- PRO Polars exploits the strong support for concurrency in Rust to run many operations in parallel. While some operations in Pandas are multi-threaded the core of the library is single-threaded and an additional library such as Dask must be used to parallelize operations:

## Reading

- https://www.datacamp.com/blog/an-introduction-to-polars-python-s-tool-for-large-scale-data-analysis
- 

